# Aprendizaje no supervisado

Los datos **no** están **etiquetados**. 

No hay una variable que predecir.

El objetivo es buscar elementos similares. 

**NO hay una variable objetivo a predecir**

Ejemplos:
  * Segmentación de clientes.
  * Jugadores similares del FIFA.
  * Compradores del supermercado semejantes.
  * Estaciones de BICIMAD similares.

<img width=800 src="https://www.researchgate.net/publication/354960266/figure/fig1/AS:1075175843983363@1633353305883/The-main-types-of-machine-learning-Main-approaches-include-classification-and.png">

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

## Clustering

Proceso de dividir un conjunto de objetos en varios grupos, de manera que los objetos en el mismo grupo (un cluster) sean más similares entre sí que con los de otros grupos.

<img width=600 src="https://media.geeksforgeeks.org/wp-content/uploads/merge3cluster.jpg">

### K-Means

Este algoritmo divide nuestros datos en K clusters buscando centroides y calculando distancias.

Minimiza un criterio conocido como la inercia o la suma de cuadrados dentro del cluster

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
from sklearn.datasets import load_iris
data = load_iris()
df = pd.DataFrame(data["data"], columns=data["feature_names"])

In [ ]:
df.head()

Ejercicio: cuál es la distancia entre la planta 0 y 1?

Dado que KMeans se basa en la **distancia** y nuestras variables tienen magnitudes muy diferentes, 

debemos **estandarizar los datos**

In [ ]:
scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [ ]:
df.head()

In [ ]:
km = KMeans(n_clusters=3, n_init="auto")
km.fit(df)

In [ ]:
cluster_n = km.predict(df)
cluster_n

In [ ]:
sns.scatterplot(x=df["sepal length (cm)"], y=df["sepal width (cm)"], hue=cluster_n)

### DBSCAN

**No** definimos el número de clusters a priori

Definimos la distancia máxima de un punto a otro para que conecten. La distancia de un elemento a otro de un cluster puede ser mayor.

El número de clusters resultante dependerá de los datos.

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
dbscan = DBSCAN(eps=0.5)

In [ ]:
dbscan.fit(df)

In [ ]:
sns.scatterplot(x=df["sepal length (cm)"], y=df["sepal width (cm)"], hue=dbscan.labels_)

### Más métodos

[sklearn documentation Clustering](https://scikit-learn.org/stable/modules/clustering.html)

<img width=600 src="https://scikit-learn.org/stable/_images/sphx_glr_plot_cluster_comparison_001.png">

## Clustering: métricas

Una métrica mide cómo de bien clusterizados están los datos

### Silhouette Score

La Silhouette Score es una métrica de la buena separación entre clusters.

Va de -1 a 1, donde:
 - los valores negativos significan que los cluster están mal asignados y se solapan mucho
 - 0 significa que los conglomerados se solapan un poco
 - los valores positivos indican que los cluster están bien separados y definidos.

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
silhouette_score(df, cluster_n)

Podemos lanzar varias pruebas y ver el K óptimo

In [ ]:
for k in range(2, 10):
    km = KMeans(n_clusters=k, n_init="auto")
    km.fit(df)
    
    cluster_n = km.predict(df)
    
    score = silhouette_score(df, cluster_n)

    print(f"K={k}, score={round(score, 3)}")

Veamos el de 2 clusters

In [ ]:
km = KMeans(n_clusters=2, n_init="auto")
km.fit(df)

In [ ]:
cluster_n = km.predict(df)

In [ ]:
sns.scatterplot(x=df["sepal length (cm)"], y=df["sepal width (cm)"], hue=cluster_n)

In [ ]:
silhouette_score(df, dbscan.labels_)

### Test del codo

<img width=500 src="https://www.oreilly.com/api/v2/epubs/9781788295758/files/assets/995b8b58-06f1-4884-a2a1-f3648428e947.png">

inertia = distancia media al centro del cluster

In [ ]:
inertias = []

for k in range(2, 20):
    km = KMeans(n_clusters=k, n_init="auto")
    km.fit(df)

    inertia = km.inertia_
    inertias.append(inertia)

In [ ]:
plt.plot(inertias)
plt.xticks(range(1,21));

## Caso de uso: tickets de supermercado

Quiero identificar tickets similares

### Preprocesado productos

In [ ]:
data = pd.read_csv("./datasets/tickets_products.csv")
data["datetime"] = data["Date"] + " " + data["Time"]
data.datetime = pd.to_datetime(data.datetime)
data = data.drop(columns=["Date", "Time"])

Cada fila es un producto de un ticket (transaction)

In [ ]:
data.shape

In [ ]:
data.head(10)

In [ ]:
data["Item"].unique()

Hay muchos valores de Item diferentes, agrupémoslos en categorías

In [ ]:
groups = {
    "beverage":['Hot chocolate', 'Coffee', 'Tea', 'Mineral water', 'Juice', 'Coke', 'Smoothies'],
    "kids":["Ella's Kitchen Pouches", 'My-5 Fruit Shoot', 'Kids biscuit'],
    "snacks":['Mighty Protein', 'Pick and Mix Bowls', 'Caramel bites', 'Bare Popcorn', 'Crisps', 'Cherry me Dried fruit', 'Raw bars'],
    "bread":['Bread', 'Toast', 'Baguette', 'Focaccia', 'Scandinavian'],
    "breakfast_pastry":['Muffin', 'Pastry', 'Medialuna', 'Scone'],
    "dessert":['Cookies', 'Tartine', 'Fudge', 'Victorian Sponge', 'Cake', 'Alfajores', 'Brownie', 'Bread Pudding', 'Bakewell', 'Raspberry shortbread sandwich', 'Lemon and coconut', 'Crepes', 'Chocolates', 'Truffles', 'Panatone'],
    "condiments":['Jam', 'Dulce de Leche', 'Honey', 'Gingerbread syrup', 'Extra Salami or Feta', 'Bacon', 'Spread', 'Chimichurri Oil'],
    "breakfast":['Eggs', 'Frittata', 'Granola', 'Muesli', 'Duck egg', 'Brioche and salami'],
    "lunch":['Soup', 'Sandwich', 'Chicken sand', 'Salad', 'Chicken Stew']
}

In [ ]:
def category(product):
    for k, v in groups.items():
        if product in v:
            return k
            
    return "other"

In [ ]:
category("Coffee")

In [ ]:
category("Granola")

In [ ]:
data["category"] = data["Item"].apply(category)
data = data.drop(columns="Item")

In [ ]:
data.head()

In [ ]:
data["category"].unique()

get_dummies

In [ ]:
data_orig = data.copy()

In [ ]:
data = pd.get_dummies(data, columns=["category"], prefix="type")

In [ ]:
data.head()

In [ ]:
data.shape

### Agrupando por ticket

In [ ]:
data.head()

Sin tener en cuenta la fecha...

In [ ]:
tickets = data.drop("datetime", axis=1).groupby("Transaction").sum()

In [ ]:
tickets.shape

In [ ]:
tickets.head()

### KMeans

Encontremos tickets similares

In [ ]:
km = KMeans(n_clusters=3, n_init="auto")

In [ ]:
km.fit(tickets)

In [ ]:
km.labels_

In [ ]:
silhouette_score(tickets, km.labels_)

In [ ]:
data_orig.head()

In [ ]:
cluster_n = pd.Series(km.labels_, index=tickets.index)

In [ ]:
data_orig["cluster_n"] = data_orig.Transaction.map(cluster_n)

In [ ]:
pd.crosstab(data_orig.category, data_orig.cluster_n)

In [ ]:
sns.heatmap(pd.crosstab(data_orig.category, data_orig.cluster_n))

En la división anterior no hay grupos claramente diferenciados

Probemos a utilizar datos de tiempo!

### KMeans incluyendo date time

In [ ]:
transaction_dates = data.groupby("Transaction").datetime.first()
tickets["hour"] = transaction_dates.dt.hour
tickets["day"] = transaction_dates.dt.dayofweek

In [ ]:
tickets.head()

In [ ]:
km = KMeans(n_clusters=4, n_init="auto")

In [ ]:
km.fit(tickets)

In [ ]:
km.labels_

In [ ]:
silhouette_score(tickets, km.labels_)

In [ ]:
data_orig.head()

In [ ]:
cluster_n = pd.Series(km.labels_, index=tickets.index)

In [ ]:
data_orig["cluster_n"] = data_orig.Transaction.map(cluster_n)

In [ ]:
data_orig["day"] = data_orig.datetime.dt.weekday
data_orig["hour"] = data_orig.datetime.dt.hour

In [ ]:
pd.crosstab(data_orig.day, data_orig.cluster_n)

In [ ]:
sns.heatmap(pd.crosstab(data_orig.day, data_orig.cluster_n))

In [ ]:
pd.crosstab(data_orig.hour, data_orig.cluster_n)

In [ ]:
sns.heatmap(pd.crosstab(data_orig.hour, data_orig.cluster_n))